__This notebook__ computes teacher logits for editable fine-tuning on Imagenet

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=YOURDEVICEHERE

imagenet_train_path = '../../imagenet/train'  # path to train ImageFolder
logits_path = './imagenet_logits/'            # saves logits to this path

import os, sys, time
sys.path.insert(0, '..')
import lib

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

import random
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)

import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'


env: CUDA_VISIBLE_DEVICES=5


In [2]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    imagenet_train_path,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

batch_size = 64

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False,
    num_workers=1, pin_memory=True)

In [ ]:
from tqdm import tqdm_notebook, tnrange
from IPython.display import clear_output

import torchvision

image_index = 0

def get_logits(model, X_test):
    with lib.training_mode(model, is_train=False):
        return lib.eval_func(lib.Lambda(lambda x: model(x.to(device))),
                                        X_test, device='cuda', batch_size=64)

model = torchvision.models.resnet18(pretrained=True).to(device)

for batch in tqdm_notebook(train_loader):
    logits = get_logits(model, batch[0])
    for logit in logits:
        image_name = train_loader.dataset.imgs[image_index][0]
        image_name = "{}/{}".format(logits_path, image_name[image_name.rfind("/") + 1:image_name.find(".j")])
        np.save(image_name, torch.softmax(logits, dim=-1))
        image_index += 1

In [5]:
!du -h imagenet_logits

9.9G	imagenet_logits
